# Assignment 3:
### Marketing department of Adventure Works Cycles chahti h ki hum unke new potential buyers(potential customers) find kre kon ho skte hai...patterns sikh kr dataset ke.

In [4]:
#for downloading dataset from kaggle...
import kagglehub

# Download latest version
path = kagglehub.dataset_download("jahias/microsoft-adventure-works-cycles-customer-data")

print("Path to dataset files:", path)

/Users/deeplatiyan/miniconda3/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 939k/939k [00:02<00:00, 450kB/s]

Extracting files...
Path to dataset files: /Users/deeplatiyan/.cache/kagglehub/datasets/jahias/microsoft-adventure-works-cycles-customer-data/versions/1


### yha se shuru :

In [15]:
# Step 1: Load the dataset and examine attributes
import pandas as pd

#load csv file
df = pd.read_csv('AWCustomers.csv')
df.head()#for first 5 lines to look at the columns n values to understand what attributes r present

,CustomerID,Title,FirstName,MiddleName,LastName,Suffix,AddressLine1,AddressLine2,City,StateProvinceName,...,Education,Occupation,Gender,MaritalStatus,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome,LastUpdated
0,21173,NaN,Chad,C,Yuan,NaN,7090 C. Mount Hood,NaN,Wollongong,New South Wales,...,Bachelors,Clerical,M,M,1,3,0,1,81916,2017-03-06
1,13249,NaN,Ryan,NaN,Perry,NaN,3651 Willow Lake Rd,NaN,Shawnee,British Columbia,...,Partial College,Clerical,M,M,1,2,1,2,81076,2017-03-06
2,29350,NaN,Julia,NaN,Thompson,NaN,1774 Tice Valley Blvd.,NaN,West Covina,California,...,Bachelors,Clerical,F,S,0,3,0,0,86387,2017-03-06
3,13503,NaN,Theodore,NaN,Gomez,NaN,2103 Baldwin Dr,NaN,Liverpool,England,...,Partial College,Skilled Manual,M,M,1,2,1,2,61481,2017-03-06
4,22803,NaN,Marshall,J,Shan,NaN,Am Gallberg 234,NaN,Werne,Nordrhein-Westfalen,...,Partial College,Skilled Manual,M,S,1,1,0,0,51804,2017-03-06


In [10]:
print(df.columns) #data ke sare attributes/features/columns aa jynge
#aur fir baad m yha se select krnge konsa kaam ka feature h.
#Analyze columns to decide which features logically influence bike purchase.

Index(['CustomerID', 'Title', 'FirstName', 'MiddleName', 'LastName', 'Suffix',
       'AddressLine1', 'AddressLine2', 'City', 'StateProvinceName',
       'CountryRegionName', 'PostalCode', 'PhoneNumber', 'BirthDate',
       'Education', 'Occupation', 'Gender', 'MaritalStatus', 'HomeOwnerFlag',
       'NumberCarsOwned', 'NumberChildrenAtHome', 'TotalChildren',
       'YearlyIncome', 'LastUpdated'],
      dtype='object')


In [11]:
#Step 2: Feature selection, cleaning, and preprocessing


# Q-1,2: Exmine Attributes & Selected Relevant Features, Create New DataFrame with Selected Features.

# 2.1.new dataframe with selected attributes(features) which are helpful.
selected_cols = ['Education', 'Occupation', 'Gender', 'MaritalStatus', 'HomeOwnerFlag', 
                 'NumberCarsOwned', 'NumberChildrenAtHome', 'TotalChildren', 'YearlyIncome']

df_selected = df[selected_cols]
df_selected.head()

#Reason: Focusing on relevant variables helps algo learn real patters, not noise.

,Education,Occupation,Gender,MaritalStatus,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome
0,Bachelors,Clerical,M,M,1,3,0,1,81916
1,Partial College,Clerical,M,M,1,2,1,2,81076
2,Bachelors,Clerical,F,S,0,3,0,0,86387
3,Partial College,Skilled Manual,M,M,1,2,1,2,61481
4,Partial College,Skilled Manual,M,S,1,1,0,0,51804


In [12]:
# 2.2.Dictionary for attributes types from the selected attributes
#dekhne ke liye ki attributes ke data type kya h.

attribute_types_ki_dictionary = {
    #nomial : categories without instrinsic order(e.g. gender, occupation)
    'nominal' : [
        'CustomerID', 'Title', 'FirstName', 'MiddleName', 'LastName', 'Suffix',
        'AddressLine1', 'AddressLine2', 'City', 'StateProvinceName',
        'CountryRegionName', 'PostalCode', 'PhoneNumber', 'Occupation',
        'Gender', 'MaritalStatus', 'HomeOwnerFlag'
    ],
    
    #ordinary : categories with an order(e.g. Education)
    'ordinal' : ['Education'],
    
    #discrete : countable numeric values
    'discrete' : ['NumberCarsOwned', 'NumberChildrenAtHome', 'TotalChildren'],
    
    #continuous : values that can take any in any range
    'continous' : ['YearlyIncome']
    
    #ratio : it has true value, and can't be zero
    
    #interval : it does'nt hv true zero, & it can be -ve values
}

print(attribute_types_ki_dictionary)

#Reason: Choosing the correct type helps select suitable preprocessing and modeling techniques.

{'nominal': ['CustomerID', 'Title', 'FirstName', 'MiddleName', 'LastName', 'Suffix', 'AddressLine1', 'AddressLine2', 'City', 'StateProvinceName', 'CountryRegionName', 'PostalCode', 'PhoneNumber', 'Occupation', 'Gender', 'MaritalStatus', 'HomeOwnerFlag'], 'ordinal': ['Education'], 'discrete': ['NumberCarsOwned', 'NumberChildrenAtHome', 'TotalChildren'], 'continous': ['YearlyIncome']}


In [8]:
#Step 3: Buyer vs Non_buyer Analysis
#Q-2: Is there a significant difference in average income?
#Theory: Compare income of buyers vs non-buyers using mean and a t-test(statistical test for difference in means)

from scipy.stats import ttest_ind

buyers = df_selected[df_selected['BikeBuyer'] == 1]['YearlyIncome']
nonbuyers = df_selected[df_selected['BikeBuyer'] == 0]['YearlyIncome']


print("Buyers abg income:", buyers.mean())
print("non-buyers avg income:", nonbuyers.mean())

tt_stat, p_val = ttest_ind(buyers, nonbuyers, equal_var=False)
print(f"T-test p-value: {p_val}")

#p-value <0.05 means significant difference.

KeyError: 'BikeBuyer'